Os efeitos de status socioeconomico sobre obesidade - Thuanny Helen, Fredson Arthur e Maria Eduarda Santos

In [ ]:
import time
import traceback
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import randint, uniform  # Distribuições estatísticas
import re
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    cross_val_score,
    KFold
)
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    make_scorer
)
import joblib
from sklearn import set_config

# Configurar estilo de plots
sns.set(style="whitegrid", palette="muted", font_scale=1.1)
plt.rcParams["figure.figsize"] = (12, 6)

# Configurar warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv(
    'Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv')

display(df)

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Total,Age(years),Education,Gender,Income,Race/Ethnicity,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,32.0,32.0,NaN,NaN,30.5,33.5,7304.0,Total,NaN,NaN,NaN,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1,Total,Total,OVR,OVERALL
1,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,32.3,32.3,NaN,NaN,29.9,34.7,2581.0,NaN,NaN,NaN,Male,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1,Gender,Male,GEN,MALE
2,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,31.8,31.8,NaN,NaN,30.0,33.6,4723.0,NaN,NaN,NaN,Female,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1,Gender,Female,GEN,FEMALE
3,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,33.6,33.6,NaN,NaN,29.9,37.6,1153.0,NaN,NaN,Less than high school,NaN,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1,Education,Less than high school,EDU,EDUHS
4,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,32.8,32.8,NaN,NaN,30.2,35.6,2402.0,NaN,NaN,High school graduate,NaN,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1,Education,High school graduate,EDU,EDUHSGRAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53387,2016,2016,VI,Virgin Islands,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,NaN,NaN,~,Data not available because sample size is insu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Asian,"(18.335765, -64.896335)",PA,PA1,Q047,VALUE,78,Race/Ethnicity,Asian,RACE,RACEASN
53388,2016,2016,VI,Virgin Islands,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,NaN,NaN,~,Data not available because sample size is insu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hawaiian/Pacific Islander,"(18.335765, -64.896335)",PA,PA1,Q047,VALUE,78,Race/Ethnicity,Hawaiian/Pacific Islander,RACE,RACEHPI
53389,2016,2016,VI,Virgin Islands,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,NaN,NaN,~,Data not available because sample size is insu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Indian/Alaska Native,"(18.335765, -64.896335)",PA,PA1,Q047,VALUE,78,Race/Ethnicity,American Indian/Alaska Native,RACE,RACENAA
53390,2016,2016,VI,Virgin Islands,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,NaN,NaN,~,Data not available because sample size is insu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 or more races,"(18.335765, -64.896335)",PA,PA1,Q047,VALUE,78,Race/Ethnicity,2 or more races,RACE,RACE2PLUS


In [ ]:
##PASSO: 6 modelos selecionados para otimização: ['Linear Regression', 'Gradient Boosting', 'Random Forest']

# Configurações iniciais
warnings.filterwarnings('ignore')
#pd.set_option('display.max_columns', 50)
sns.set(style='whitegrid', palette='pastel')
plt.rcParams['font.size'] = 12

def clean_column_names(df):
    """Padroniza nomes de colunas"""
    df.columns = [re.sub(r'[^a-zA-Z0-9_]', '', col.lower().replace(' ', '_'))
                 for col in df.columns]
    return df

def basic_data_cleaning(df):
    """Realiza limpeza básica dos dados"""
    df = clean_column_names(df)

    # Identifica coluna alvo
    target_candidates = ['datavalue', 'value', 'obesityrate', 'prevalence', 'rate']
    for col in target_candidates:
        if col in df.columns:
            df.rename(columns={col: 'obesity_rate'}, inplace=True)
            break
    else:
        # Se não encontrar, assume a primeira coluna numérica como alvo
        num_cols = df.select_dtypes(include=np.number).columns
        if len(num_cols) > 0:
            df.rename(columns={num_cols[0]: 'obesity_rate'}, inplace=True)
        else:
            # Se não houver colunas numéricas, cria uma coluna vazia
            df['obesity_rate'] = np.nan

    # Remove colunas completamente vazias
    df.dropna(axis=1, how='all', inplace=True)

    # Remove linhas duplicadas
    df = df.drop_duplicates()

    return df

class DataPreprocessor(BaseEstimator, TransformerMixin):
    """Pipeline personalizado para pré-processamento"""
    def _init_(self, date_columns=None, categorical_threshold=15):
        self.date_columns = date_columns
        self.categorical_threshold = categorical_threshold

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Tratamento de datas
        if self.date_columns:
            for col in self.date_columns:
                if col in X.columns:
                    try:
                        X[col] = pd.to_datetime(X[col], errors='coerce')
                        X[f'{col}_year'] = X[col].dt.year.fillna(X[col].dt.year.median())
                        X[f'{col}_month'] = X[col].dt.month.fillna(X[col].dt.month.median())
                    except Exception as e:
                        print(f"Erro ao processar coluna de data {col}: {str(e)}")

        # Conversão de tipos
        for col in X.select_dtypes(include='object').columns:
            # Tenta converter para numérico
            try:
                X[col] = pd.to_numeric(X[col], errors='ignore')
            except:
                pass

            # Identifica colunas categóricas com baixa cardinalidade
            unique_count = X[col].nunique()
            if 1 < unique_count < self.categorical_threshold:
                X[col] = X[col].astype('category')

        # Tratamento de outliers para colunas numéricas
        num_cols = X.select_dtypes(include=np.number).columns
        for col in num_cols:
            # Ignora colunas com baixa variância
            if X[col].nunique() > 1:
                q1 = X[col].quantile(0.05)
                q3 = X[col].quantile(0.95)
                X[col] = np.clip(X[col], q1, q3)

        return X

def feature_engineering(df):
    """Cria novas features a partir dos dados existentes"""
    if df is None:
        return None

    try:
        df = df.copy()

        # 1. Conversão segura de colunas de ano
        for col in ['yearstart', 'yearend']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # 2. Média de anos (depois da conversão)
        if 'yearstart' in df.columns and 'yearend' in df.columns:
            df['year_avg'] = (df['yearstart'] + df['yearend']) / 2

        # 3. Mapeamento de regiões
        if 'locationdesc' in df.columns:
            regions = {
                'northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire',
                              'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania'],
                'midwest': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin',
                            'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'],
                'south': ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina',
                          'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia',
                          'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
                'west': ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming',
                         'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
            }

            def map_region(location):
                for region, states in regions.items():
                    if location in states:
                        return region
                return 'other'

            df['region'] = df['locationdesc'].apply(map_region)

        # 4. Preenchimento de valores ausentes nas colunas temporais
        for col in ['yearstart', 'yearend', 'year_avg']:
            if col in df.columns:
                if df[col].isnull().sum() > 0:
                    median_val = df[col].median()
                    df[col].fillna(median_val, inplace=True)

        # 5. Agrupamento de categorias raras
        if 'stratificationcategory1' in df.columns:
            category_counts = df['stratificationcategory1'].value_counts()
            rare_categories = category_counts[category_counts < 100].index
            df['strat_group'] = df['stratificationcategory1'].replace(rare_categories, 'Other')

        return df

    except Exception as e:
        print(f"Erro na engenharia de features: {str(e)}")
        traceback.print_exc()
        return df


In [ ]:
# ====================================================
# PASSO: 3 Visualização de Dados
# ====================================================
def create_diagnostic_plots(df):
    """Gera visualizações diagnósticas dos dados"""
    if df is None or 'obesity_rate' not in df.columns:
        print("Dados inválidos ou coluna 'obesity_rate' não encontrada para plotagem")
        return

    try:
        # Conversões seguras
        df['obesity_rate'] = pd.to_numeric(df['obesity_rate'], errors='coerce')
        if 'yearstart' in df.columns:
            df['yearstart'] = pd.to_numeric(df['yearstart'], errors='coerce')

        plt.figure(figsize=(18, 14))

        # 1. Distribuição da variável alvo
        plt.subplot(2, 2, 1)
        sns.histplot(df['obesity_rate'].dropna(), kde=True, bins=20)
        plt.title('Distribuição da Taxa de Obesidade', fontsize=14)
        plt.xlabel('Taxa de Obesidade (%)', fontsize=12)
        plt.ylabel('Frequência', fontsize=12)

        # 2. Evolução temporal
        plt.subplot(2, 2, 2)
        if 'yearstart' in df.columns and df['yearstart'].nunique() > 1:
            yearly_data = df.groupby('yearstart')['obesity_rate'].agg(['mean', 'median', 'std']).dropna()
            plt.plot(yearly_data.index, yearly_data['median'], marker='o', linestyle='-', color='b')
            plt.fill_between(
                yearly_data.index,
                yearly_data['median'] - yearly_data['std'],
                yearly_data['median'] + yearly_data['std'],
                alpha=0.2, color='b'
            )
            plt.title('Evolução Anual da Obesidade', fontsize=14)
            plt.xlabel('Ano', fontsize=12)
            plt.ylabel('Taxa de Obesidade (Mediana)', fontsize=12)
            plt.grid(True, linestyle='--', alpha=0.7)
        else:
            plt.text(0.5, 0.5, 'Dados insuficientes para análise temporal',
                     horizontalalignment='center', verticalalignment='center')
            plt.title('Evolução Anual da Obesidade', fontsize=14)

        # 3. Obesidade por categoria
        plt.subplot(2, 2, 3)
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        cat_col = None
        for col in categorical_cols:
            if col != 'locationdesc' and 1 < df[col].nunique() <= 20:
                cat_col = col
                break
        if cat_col is None and len(categorical_cols) > 0:
            cat_col = categorical_cols[0]

        if cat_col:
            if df[cat_col].nunique() > 10:
                top_cats = df[cat_col].value_counts().nlargest(10).index
                filtered_df = df[df[cat_col].isin(top_cats)]
            else:
                filtered_df = df

            if not filtered_df.empty and filtered_df['obesity_rate'].notnull().any():
                sns.boxplot(
                    x=cat_col,
                    y='obesity_rate',
                    data=filtered_df.dropna(subset=['obesity_rate', cat_col])
                )
                plt.xticks(rotation=45, ha='right')
                plt.title(f'Obesidade por {cat_col}', fontsize=14)
                plt.xlabel(cat_col, fontsize=12)
                plt.ylabel('Taxa de Obesidade', fontsize=12)
            else:
                plt.text(0.5, 0.5, 'Dados insuficientes para plotar',
                         horizontalalignment='center', verticalalignment='center')
                plt.title(f'Obesidade por {cat_col}', fontsize=14)
        else:
            plt.text(0.5, 0.5, 'Nenhuma coluna categórica disponível',
                     horizontalalignment='center', verticalalignment='center')
            plt.title('Obesidade por Categoria', fontsize=14)

        # 4. Matriz de correlação
        plt.subplot(2, 2, 4)
        numeric_df = df.select_dtypes(include=np.number)
        if numeric_df.shape[1] > 1:
            corr_matrix = numeric_df.corr()
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
            sns.heatmap(
                corr_matrix,
                mask=mask,
                annot=True,
                fmt=".2f",
                cmap='coolwarm',
                vmin=-1,
                vmax=1,
                annot_kws={"size": 8}
            )
            plt.title('Matriz de Correlação', fontsize=14)
        else:
            plt.text(0.5, 0.5, 'Dados numéricos insuficientes para matriz de correlação',
                     horizontalalignment='center', verticalalignment='center')
            plt.title('Matriz de Correlação', fontsize=14)

        plt.tight_layout()
        plt.savefig('analise_obesidade_diagnostico.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Visualizações salvas como 'analise_obesidade_diagnostico.png'")

    except Exception as e:
        print(f"❌ Erro ao gerar visualizações: {str(e)}")
        traceback.print_exc()

In [ ]:
# ====================================================
# PASSO: 4 Modelagem Preditiva
# ====================================================
def build_model_pipeline(X_train):
    """Constrói pipeline de modelagem dinâmico baseado nos dados"""
    # Identifica features
    features = [col for col in X_train.columns]

    # Separa features numéricas e categóricas
    numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    # Transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Pré-processador
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Modelo base
    model = GradientBoostingRegressor(
        random_state=42,
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=10
    )

    # Pipeline completo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    return pipeline, numeric_features, categorical_features

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import traceback
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import randint, uniform
import joblib

# ====================================================
# PASSO: 5 Fluxo Principal de Execução
# ====================================================
def basic_data_cleaning(df):
    """Realiza passos básicos de limpeza de dados."""
    print("Realizando limpeza básica de dados...")
    df.columns = [re.sub(r'[^a-zA-Z0-9_]', '', col.lower().replace(' ', '_'))
                 for col in df.columns]

    target_candidates = ['datavalue', 'value', 'obesityrate', 'prevalence', 'rate']
    for col in target_candidates:
        if col in df.columns:
            df.rename(columns={col: 'obesity_rate'}, inplace=True)
            break
    else:
        num_cols = df.select_dtypes(include=np.number).columns
        if len(num_cols) > 0:
            df.rename(columns={num_cols[0]: 'obesity_rate'}, inplace=True)
        else:
            df['obesity_rate'] = np.nan

    threshold = len(df) * 0.5
    df_cleaned = df.dropna(axis=1, thresh=threshold)
    print(f"Colunas originais: {df.shape[1]}, Colunas após limpeza: {df_cleaned.shape[1]}")
    return df_cleaned

class DataPreprocessor(BaseEstimator, TransformerMixin):
    """Transformer customizado para pré-processamento de dados."""
    def __init__(self, date_columns=None, categorical_threshold=20):
        self.date_columns = date_columns or []
        self.categorical_threshold = categorical_threshold
        self.categorical_features = None
        self.numeric_features = None

    def fit(self, X, y=None):
        self.categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        self.numeric_features = X.select_dtypes(include=np.number).columns.tolist()
        return self

    def transform(self, X):
        print("Realizando pré-processamento de dados...")
        X_processed = X.copy()

        for col in self.date_columns:
            if col in X_processed.columns:
                try:
                    X_processed[col] = pd.to_datetime(X_processed[col], errors='coerce')
                    X_processed[col + '_year'] = X_processed[col].dt.year
                    X_processed = X_processed.drop(columns=[col])
                    print(f"Coluna de data processada: {col}")
                except Exception as e:
                    print(f"Não foi possível processar a coluna de data {col}: {e}")

        for col in self.categorical_features:
            if col in X_processed.columns:
                if X_processed[col].nunique() > self.categorical_threshold:
                    print(f"Aviso: A coluna categórica '{col}' tem muitos valores únicos ({X_processed[col].nunique()}).")

        num_cols = X_processed.select_dtypes(include=np.number).columns
        for col in num_cols:
            if X_processed[col].nunique() > 1:
                q1 = X_processed[col].quantile(0.05)
                q3 = X_processed[col].quantile(0.95)
                X_processed[col] = np.clip(X_processed[col], q1, q3)

        print("Pré-processamento concluído.")
        return X_processed

def feature_engineering(df):
    """Realiza engenharia de features."""
    print("Realizando engenharia de features...")
    df_engineered = df.copy()

    if 'yearstart_year' in df_engineered.columns and 'yearend_year' in df_engineered.columns:
        df_engineered['year_difference'] = df_engineered['yearend_year'] - df_engineered['yearstart_year']
        print("Criada feature 'year_difference'.")

    # Remover qualquer feature que possa conter a variável alvo
    forbidden_features = ['obesity_rate', 'data_value', 'value', 'rate', 'prevalence']
    for col in forbidden_features:
        if col in df_engineered.columns:
            print(f"⚠️ Removendo possível vazamento: {col}")
            df_engineered = df_engineered.drop(columns=[col])

    print("Engenharia de features concluída.")
    return df_engineered

def compare_models(X_train, y_train, X_test, y_test, preprocessor_pipe, sample_size=5000):
    """Compara a performance de diferentes modelos de regressão com amostragem."""
    print("\nComparando diferentes modelos com amostra de dados...")

    # Amostrar dados para acelerar o processo
    if len(X_train) > sample_size:
        X_train_sample = X_train.sample(sample_size, random_state=42)
        y_train_sample = y_train.loc[X_train_sample.index]
        print(f"Usando amostra de {sample_size} registros para comparação de modelos")
    else:
        X_train_sample = X_train
        y_train_sample = y_train
        print("Usando conjunto completo de treino para comparação de modelos")

    models = {
        'Linear Regression': LinearRegression(n_jobs=-1),
        'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'ElasticNet': ElasticNet(random_state=42),
        'KNeighbors Regressor': KNeighborsRegressor(n_jobs=-1)
    }
    results = []

    for name, model in models.items():
        try:
            start_time = time.time()
            pipeline = Pipeline(steps=[('preprocessor', preprocessor_pipe),
                                      ('regressor', model)])
            pipeline.fit(X_train_sample, y_train_sample)
            y_pred = pipeline.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            train_time = time.time() - start_time

            # Formatar resultados para mostrar números reais
            results.append({
                'Modelo': name,
                'RMSE': f"{rmse:.4f}",
                'R²': f"{r2:.4f}",
                'MAE': f"{mae:.4f}",
                'Tempo Treino (s)': f"{train_time:.2f}"
            })
            print(f"  - {name}: RMSE={rmse:.4f}, R²={r2:.4f}, MAE={mae:.4f}, Tempo={train_time:.2f}s")
        except Exception as e:
            print(f"  - Erro ao treinar/avaliar {name}: {str(e)}")
            results.append({
                'Modelo': name,
                'RMSE': "Erro",
                'R²': "Erro",
                'MAE': "Erro",
                'Tempo Treino (s)': "Erro"
            })

    results_df = pd.DataFrame(results).sort_values('RMSE')
    print("\nResultados da comparação de modelos:")
    print(results_df)

    try:
        # Converter para numérico para plotagem
        plot_df = results_df.copy()
        plot_df['RMSE'] = pd.to_numeric(plot_df['RMSE'], errors='coerce')
        plot_df['R²'] = pd.to_numeric(plot_df['R²'], errors='coerce')
        plot_df['MAE'] = pd.to_numeric(plot_df['MAE'], errors='coerce')
        plot_df['Tempo Treino (s)'] = pd.to_numeric(plot_df['Tempo Treino (s)'], errors='coerce')

        plt.figure(figsize=(12, 8))
        plt.subplot(2, 2, 1)
        sns.barplot(x='RMSE', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Comparação de RMSE')
        plt.xlabel('RMSE')
        plt.subplot(2, 2, 2)
        sns.barplot(x='R²', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Comparação de R²')
        plt.xlabel('Coeficiente de Determinação (R²)')
        plt.subplot(2, 2, 3)
        sns.barplot(x='MAE', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Comparação de MAE')
        plt.xlabel('Erro Absoluto Médio (MAE)')
        plt.subplot(2, 2, 4)
        sns.barplot(x='Tempo Treino (s)', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Tempo de Treinamento')
        plt.xlabel('Segundos')
        plt.tight_layout()
        plt.savefig('comparacao_modelos.png', dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"Erro ao gerar gráficos: {str(e)}")

    return results_df

def optimize_hyperparameters(base_model, param_grid, preprocessor_pipe, X_train, y_train):
    """Otimiza hiperparâmetros para um dado modelo usando RandomizedSearchCV com configuração eficiente."""
    model_name = type(base_model).__name__
    print(f"Otimizando hiperparâmetros para {model_name}...")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor_pipe),
                               ('regressor', base_model)])

    if not param_grid and model_name != 'LinearRegression':
        print("Nenhuma grade de parâmetros fornecida para otimização. Usando o modelo base.")
        return pipeline.fit(X_train, y_train), {}
    elif model_name == 'LinearRegression':
        print("Linear Regression não requer otimização de hiperparâmetros. Usando o modelo base.")
        return pipeline.fit(X_train, y_train), {}

    try:
        # Configuração otimizada para velocidade
        random_search = RandomizedSearchCV(
            pipeline,
            param_grid,
            n_iter=10,  # Reduzido para 10 iterações
            cv=KFold(n_splits=3, shuffle=True, random_state=42),  # Reduzido para 3 folds
            scoring='neg_root_mean_squared_error',
            random_state=42,
            n_jobs=-1,  # Usar todos os núcleos disponíveis
            verbose=1
        )
        random_search.fit(X_train, y_train)
        print(f"Melhores parâmetros encontrados: {random_search.best_params_}")
        print(f"Melhor RMSE de validação cruzada: {-random_search.best_score_:.4f}")
        return random_search.best_estimator_, random_search.best_params_
    except Exception as e:
        print(f"Erro durante a otimização de hiperparâmetros: {str(e)}")
        traceback.print_exc()
        return pipeline.fit(X_train, y_train), {}

def compare_optimized_models(best_models, X_test, y_test):
    """Compara a performance de modelos otimizados."""
    print("\nComparando modelos otimizados...")
    results = []
    for name, model in best_models.items():
        try:
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            # Formatar resultados para mostrar números reais
            results.append({
                'Modelo': name,
                'RMSE': f"{rmse:.4f}",
                'R²': f"{r2:.4f}",
                'MAE': f"{mae:.4f}"
            })
            print(f"  - {name}: RMSE={rmse:.4f}, R²={r2:.4f}, MAE={mae:.4f}")
        except Exception as e:
            print(f"  - Erro ao avaliar o modelo otimizado {name}: {str(e)}")
            results.append({
                'Modelo': name,
                'RMSE': "Erro",
                'R²': "Erro",
                'MAE': "Erro"
            })

    results_df = pd.DataFrame(results).sort_values('RMSE')
    print("\nResultados da comparação de modelos otimizados:")
    print(results_df)

    try:
        # Converter para numérico para plotagem
        plot_df = results_df.copy()
        plot_df['RMSE'] = pd.to_numeric(plot_df['RMSE'], errors='coerce')
        plot_df['R²'] = pd.to_numeric(plot_df['R²'], errors='coerce')

        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        sns.barplot(x='RMSE', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Comparação de RMSE (Modelos Otimizados)')
        plt.xlabel('RMSE')
        plt.subplot(1, 2, 2)
        sns.barplot(x='R²', y='Modelo', data=plot_df, palette='viridis')
        plt.title('Comparação de R² (Modelos Otimizados)')
        plt.xlabel('Coeficiente de Determinação (R²)')
        plt.tight_layout()
        plt.savefig('comparacao_modelos_otimizados.png', dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"Erro ao gerar gráficos: {str(e)}")

    return results_df

def create_diagnostic_plots(df):
    """Gera visualizações diagnósticas dos dados"""
    if df is None or 'obesity_rate' not in df.columns:
        print("Dados inválidos ou coluna 'obesity_rate' não encontrada para plotagem")
        return

    try:
        df['obesity_rate'] = pd.to_numeric(df['obesity_rate'], errors='coerce')  # Corrigido typo aqui
        year_col = None
        if 'yearstart_year' in df.columns:
            year_col = 'yearstart_year'
        elif 'yearend_year' in df.columns:
            year_col = 'yearend_year'
        elif 'yearstart' in df.columns:
            year_col = 'yearstart'

        plt.figure(figsize=(18, 14))
        plt.subplot(2, 2, 1)
        sns.histplot(df['obesity_rate'].dropna(), kde=True, bins=20)
        plt.title('Distribuição da Taxa de Obesidade')
        plt.xlabel('Taxa de Obesidade (%)')
        plt.ylabel('Frequência')

        plt.subplot(2, 2, 2)
        if year_col and df[year_col].nunique() > 1:
            df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
            yearly_data = df.groupby(year_col)['obesity_rate'].agg(['mean', 'median', 'std']).dropna()
            plt.plot(yearly_data.index, yearly_data['median'], marker='o')
            plt.fill_between(yearly_data.index,
                             yearly_data['median'] - yearly_data['std'],
                             yearly_data['median'] + yearly_data['std'],
                             alpha=0.2)
            plt.title('Evolução Anual da Obesidade')
            plt.xlabel('Ano')
            plt.ylabel('Taxa de Obesidade (Mediana)')
        else:
            plt.text(0.5, 0.5, 'Dados insuficientes para análise temporal', ha='center')
            plt.title('Evolução Anual da Obesidade')

        # CORREÇÃO PRINCIPAL: Identação correta deste bloco
        plt.subplot(2, 2, 3)
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        cat_col = None
        for col in categorical_cols:
            if col != 'locationdesc' and 1 < df[col].nunique() <= 20:
                cat_col = col
                break

        # Identação corrigida abaixo
        if cat_col is None and len(categorical_cols) > 0:
            for col in categorical_cols:
                if 1 < df[col].nunique() <= 30:
                    cat_col = col
                    break

        if cat_col:
            if df[cat_col].nunique() > 10:
                top_cats = df[cat_col].value_counts().nlargest(10).index
                filtered_df = df[df[cat_col].isin(top_cats)]
            else:
                filtered_df = df

            if not filtered_df.empty and filtered_df['obesity_rate'].notnull().any():
                sns.boxplot(
                    x=cat_col,
                    y='obesity_rate',
                    data=filtered_df.dropna(subset=['obesity_rate', cat_col])
                )
                plt.xticks(rotation=45, ha='right')
                plt.title(f'Obesidade por {cat_col}')
                plt.xlabel(cat_col)
                plt.ylabel('Taxa de Obesidade')
            else:
                plt.text(0.5, 0.5, 'Dados insuficientes para plotar', ha='center')
                plt.title(f'Obesidade por {cat_col}')
        else:
            plt.text(0.5, 0.5, 'Nenhuma coluna categórica disponível', ha='center')
            plt.title('Obesidade por Categoria')

        plt.subplot(2, 2, 4)
        numeric_df = df.select_dtypes(include=np.number)
        if numeric_df.shape[1] > 1:
            corr_matrix = numeric_df.corr()
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
            sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',
                        vmin=-1, vmax=1, annot_kws={"size":8})
            plt.title('Matriz de Correlação')
        else:
            plt.text(0.5, 0.5, 'Dados numéricos insuficientes para matriz de correlação', ha='center')
            plt.title('Matriz de Correlação')

        plt.tight_layout()
        plt.savefig('analise_obesidade_diagnostico.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Visualizações salvas como 'analise_obesidade_diagnostico.png'")
    except Exception as e:
        print(f"❌ Erro ao gerar visualizações: {e}")
        traceback.print_exc()

def build_model_pipeline(X_train):
    """Constrói pipeline de modelagem dinâmico baseado nos dados"""
    numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

    model = GradientBoostingRegressor(
        random_state=42,
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=10
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    return pipeline, numeric_features, categorical_features

def main(df):
    """Fluxo principal de execução otimizado para velocidade"""
    try:
        # 1. Limpeza básica de dados
        df_cleaned = basic_data_cleaning(df)

        # 2. Identificar colunas de data automaticamente
        date_cols = [col for col in df_cleaned.columns
                     if 'date' in col.lower() or 'year' in col.lower() or 'time' in col.lower()]
        print(f"Colunas de data identificadas: {date_cols}")

        # 3. Pré-processamento
        preprocessor = DataPreprocessor(date_columns=date_cols, categorical_threshold=20)
        X = df_cleaned.drop(columns=['obesity_rate'], errors='ignore')
        y = df_cleaned['obesity_rate']

        # Converter para numérico se necessário
        if not pd.api.types.is_numeric_dtype(y):
            y = pd.to_numeric(y, errors='coerce')

        # 4. Engenharia de features
        X_preprocessed = preprocessor.fit_transform(X)
        X_engineered = feature_engineering(X_preprocessed)

        # Verificar vazamento de dados
        if 'obesity_rate' in X_engineered.columns:
            print("⚠️ ATENÇÃO: Variável alvo encontrada nas features. Removendo...")
            X_engineered = X_engineered.drop(columns=['obesity_rate'])

        # 5. Dividir dados (usando amostra se conjunto for muito grande)
        if len(X_engineered) > 10000:
            sample_size = min(10000, len(X_engineered))
            X_sample = X_engineered.sample(sample_size, random_state=42)
            y_sample = y.loc[X_sample.index]
            X_train, X_test, y_train, y_test = train_test_split(
                X_sample, y_sample, test_size=0.2, random_state=42)
            print(f"Usando amostra de {sample_size} registros para modelagem")
        else:
            X_train, X_test, y_train, y_test = train_test_split(
                X_engineered, y, test_size=0.2, random_state=42)

        # 6. Construir pipeline de modelagem
        model_pipeline, numeric_features, categorical_features = build_model_pipeline(X_train)
        preprocessor_pipe = model_pipeline.named_steps['preprocessor']

        # 7. Comparar modelos com amostra
        results_df = compare_models(X_train, y_train, X_test, y_test, preprocessor_pipe)

        # 8. Otimizar apenas os 2 melhores modelos
        top_models = results_df.head(2)['Modelo'].tolist()
        print(f"\nOtimizando os melhores modelos: {top_models}")

        # Grades de parâmetros simplificadas
        param_grids = {
            'Random Forest': {
                'regressor__n_estimators': randint(50, 200),
                'regressor__max_depth': [None, 5, 10],
                'regressor__min_samples_split': randint(2, 10)
            },
            'Gradient Boosting': {
                'regressor__n_estimators': randint(50, 150),
                'regressor__learning_rate': [0.01, 0.1, 0.2],
                'regressor__max_depth': randint(3, 8)
            },
            'Linear Regression': {}  # Sem otimização
        }

        best_models = {}
        for model_name in top_models:
            if model_name == 'Random Forest':
                base_model = RandomForestRegressor(random_state=42, n_jobs=-1)
            elif model_name == 'Gradient Boosting':
                base_model = GradientBoostingRegressor(random_state=42)
            elif model_name == 'Linear Regression':
                base_model = LinearRegression(n_jobs=-1)
            else:
                continue

            best_model, best_params = optimize_hyperparameters(
                base_model,
                param_grids.get(model_name, {}),
                preprocessor_pipe,
                X_train,
                y_train
            )
            best_models[model_name] = best_model

        # 9. Comparar modelos otimizados
        optimized_results = compare_optimized_models(best_models, X_test, y_test)

        # 10. Salvar o melhor modelo
        best_model_name = optimized_results.iloc[0]['Modelo']
        best_model = best_models[best_model_name]
        joblib.dump(best_model, 'best_model.pkl')
        print(f"Melhor modelo ({best_model_name}) salvo como 'best_model.pkl'")

        # 11. Visualizações diagnósticas
        create_diagnostic_plots(df_cleaned)

        print("\n" + "="*80)
        print("PROCESSO CONCLUÍDO COM SUCESSO!")
        print("="*80)

    except Exception as e:
        print("\n" + "="*80)
        print("❌ ERRO NO PROCESSO PRINCIPAL")
        print("="*80)
        print(f"Erro: {str(e)}")
        traceback.print_exc()

In [ ]:
# Definindo as funções ausentes: basic_data_cleaning, DataPreprocessor, feature_engineering, compare_models, e optimize_hyperparameters

def basic_data_cleaning(df):
    """Realiza passos básicos de limpeza de dados."""
    print("Realizando limpeza básica de dados...")
    # Exemplo de limpeza: remover colunas com muitos valores ausentes
    threshold = len(df) * 0.5
    df_cleaned = df.dropna(axis=1, thresh=threshold)
    print(f"Colunas originais: {df.shape[1]}, Colunas após limpeza: {df_cleaned.shape[1]}")
    return df_cleaned

class DataPreprocessor(BaseEstimator, TransformerMixin):
    """Transformer customizado para pré-processamento de dados."""
    def __init__(self, date_columns=None, categorical_threshold=20):
        self.date_columns = date_columns
        self.categorical_threshold = categorical_threshold
        self.categorical_features = None
        self.numeric_features = None

    def fit(self, X, y=None):
        # Identifica features categóricas e numéricas
        self.categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        self.numeric_features = X.select_dtypes(include=np.number).columns.tolist()
        return self

    def transform(self, X):
        print("Realizando pré-processamento de dados...")
        X_processed = X.copy()

        # Lida com colunas de data se especificadas
        if self.date_columns:
            for col in self.date_columns:
                if col in X_processed.columns:
                    try:
                        # Converte para objetos datetime, forçando erros
                        X_processed[col] = pd.to_datetime(X_processed[col], errors='coerce')
                        # Exemplo de extração de feature: ano
                        X_processed[col + '_year'] = X_processed[col].dt.year
                        # Remove a coluna de data original
                        X_processed = X_processed.drop(columns=[col])
                        print(f"Coluna de data processada: {col}")
                    except Exception as e:
                        print(f"Não foi possível processar a coluna de data {col}: {e}")

        # Lida com features categóricas (exemplo: codificação simples de rótulo ou one-hot baseada no limiar)
        if self.categorical_features:
             for col in self.categorical_features:
                 if col in X_processed.columns:
                    if X_processed[col].nunique() > self.categorical_threshold:
                        # Muitos valores únicos, talvez remover ou aplicar uma estratégia diferente
                        print(f"Aviso: A coluna categórica '{col}' tem muitos valores únicos ({X_processed[col].nunique()}). Considere tratamento alternativo.")
                        # Exemplo: Remover coluna ou manter como está por enquanto, dependendo da estratégia
                        # X_processed = X_processed.drop(columns=[col]) # Exemplo: removendo alta cardinalidade
                    else:
                         # Para demonstração, vamos mantê-las como estão antes do OneHotEncoding no pipeline do modelo
                         pass # OneHotEncoding é tratado no pipeline do modelo

        print("Pré-processamento concluído.")
        return X_processed


def feature_engineering(df):
    """Realiza engenharia de features."""
    print("Realizando engenharia de features...")
    df_engineered = df.copy()

    # Exemplo de engenharia de features: diferença entre ano de início e fim
    if 'yearstart_year' in df_engineered.columns and 'yearend_year' in df_engineered.columns:
        df_engineered['year_difference'] = df_engineered['yearend_year'] - df_engineered['yearstart_year']
        print("Criada feature 'year_difference'.")

    # Exemplo: Termo de interação (se colunas relevantes existirem e forem numéricas)
    # Verifica se as colunas 'Data_Value' e 'Sample_Size' são numéricas antes de multiplicar
    if all(col in df_engineered.columns and pd.api.types.is_numeric_dtype(df_engineered[col]) for col in ['Data_Value', 'Sample_Size']):
        df_engineered['value_sample_interaction'] = df_engineered['Data_Value'] * df_engineered['Sample_Size']
        print("Criada feature 'value_sample_interaction'.")
    else:
        # Lida com potenciais tipos não numéricos, se necessário, por exemplo, convertendo ou pulando a interação
         if 'Data_Value' in df_engineered.columns and 'Sample_Size' in df_engineered.columns:
            print("Pulando a criação da feature 'value_sample_interaction' devido a tipos não numéricos ou colunas ausentes.")


    print("Engenharia de features concluída.")
    return df_engineered

def compare_models(X_train, y_train, X_test, y_test, preprocessor_pipe):
    """Compara a performance de diferentes modelos de regressão."""
    print("\nComparando diferentes modelos...")
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'Support Vector Machine': SVR(kernel='rbf'),
        'ElasticNet': ElasticNet(random_state=42),
        'KNeighbors Regressor': KNeighborsRegressor()

    }

    results = []

    for name, model in models.items():
        try:
            # Constrói um pipeline para cada modelo usando o preprocessor fornecido
            pipeline = Pipeline(steps=[('preprocessor', preprocessor_pipe),
                                       ('regressor', model)])

            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            results.append({'Modelo': name, 'RMSE': rmse, 'R²': r2, 'MAE': mae})
            print(f"  - {name}: RMSE={rmse:.4f}, R²={r2:.4f}, MAE={mae:.4f}")

        except Exception as e:
            print(f"  - Erro ao treinar/avaliar {name}: {str(e)}")
            results.append({'Modelo': name, 'RMSE': np.nan, 'R²': np.nan, 'MAE': np.nan})


    results_df = pd.DataFrame(results).sort_values('RMSE')
    print("\nResultados da comparação de modelos:")
    display(results_df)
    return results_df

def optimize_hyperparameters(base_model, param_grid, preprocessor_pipe, X_train, y_train):
    """Otimiza hiperparâmetros para um dado modelo usando RandomizedSearchCV."""
    print(f"Otimizando hiperparâmetros para {type(base_model).__name__}...")

    # Cria um pipeline com o preprocessor e o modelo base
    pipeline = Pipeline(steps=[('preprocessor', preprocessor_pipe),
                               ('regressor', base_model)])

    if not param_grid:
        print("Nenhuma grade de parâmetros fornecida para otimização. Usando o modelo base.")
        return pipeline.fit(X_train, y_train), {}

    try:
        # Usa RandomizedSearchCV
        random_search = RandomizedSearchCV(
            pipeline,
            param_grid,
            n_iter=20, # Número de configurações de parâmetros a serem amostradas
            cv=KFold(n_splits=5, shuffle=True, random_state=42), # Usando validação cruzada KFold
            scoring='neg_root_mean_squared_error', # Otimiza para RMSE
            random_state=42,
            n_jobs=-1, # Usa todos os núcleos disponíveis
            verbose=1
        )

        random_search.fit(X_train, y_train)

        print(f"Melhores parâmetros encontrados: {random_search.best_params_}")
        print(f"Melhor RMSE de validação cruzada: {-random_search.best_score_:.4f}") # Observação: é o RMSE negativo

        return random_search.best_estimator_, random_search.best_params_

    except Exception as e:
        print(f"Erro durante a otimização de hiperparâmetros: {str(e)}")
        traceback.print_exc()
        return pipeline.fit(X_train, y_train), {} # Retorna o pipeline do modelo base se a otimização falhar

def compare_optimized_models(best_models, X_test, y_test):
    """Compara a performance de modelos otimizados."""
    print("\nComparando modelos otimizados...")
    results = []

    for name, model in best_models.items():
        try:
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            results.append({'Modelo': name, 'RMSE': rmse, 'R²': r2, 'MAE': mae})
            print(f"  - {name}: RMSE={rmse:.4f}, R²={r2:.4f}, MAE={mae:.4f}")

        except Exception as e:
            print(f"  - Erro ao avaliar o modelo otimizado {name}: {str(e)}")
            results.append({'Modelo': name, 'RMSE': np.nan, 'R²': np.nan, 'MAE': np.nan})


    results_df = pd.DataFrame(results).sort_values('RMSE')
    print("\nResultados da comparação de modelos otimizados:")
    display(results_df)
    return results_df

In [ ]:
# ====================================================
# PASSO: 7 Comparação de Modelos
# ====================================================
def compare_models(X_train, y_train, X_test, y_test, preprocessor):
    """Treina e compara múltiplos modelos de regressão"""
    models = {
        'Gradient Boosting': GradientBoostingRegressor(
            random_state=42, n_estimators=150, learning_rate=0.1, max_depth=5, min_samples_split=10
        ),
        'Random Forest': RandomForestRegressor(
            random_state=42, n_estimators=100, max_depth=None, min_samples_split=2
        ),
        'Linear Regression': LinearRegression(),
        'Support Vector Machine': SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1),
        'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=5),
        'ElasticNet': ElasticNet(random_state=42, alpha=0.1, l1_ratio=0.5)
    }

    results = []

    for name, model in models.items():
        try:
            start_time = time.time()

            # Criar pipeline completo
            pipeline = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('regressor', model)
            ])

            # Treinar modelo
            pipeline.fit(X_train, y_train)

            # Fazer previsões
            y_pred = pipeline.predict(X_test)

            # Calcular métricas
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            # Tempo de execução
            train_time = time.time() - start_time

            # Armazenar resultados
            results.append({
                'Modelo': name,
                'RMSE': rmse,
                'R²': r2,
                'MAE': mae,
                'MSE': mse,
                'Tempo Treino (s)': train_time
            })

            print(f"{name}: RMSE={rmse:.4f}, R²={r2:.4f}, MAE={mae:.4f}, Tempo={train_time:.2f}s")

        except Exception as e:
            print(f"Erro ao treinar {name}: {str(e)}")

    # Criar DataFrame de resultados
    results_df = pd.DataFrame(results).sort_values('RMSE')

    # Plotar comparação de desempenho
    plt.figure(figsize=(12, 8))

    # RMSE
    plt.subplot(2, 2, 1)
    sns.barplot(x='RMSE', y='Modelo', data=results_df, palette='viridis')
    plt.title('Comparação de RMSE')
    plt.xlabel('RMSE')

    # R²
    plt.subplot(2, 2, 2)
    sns.barplot(x='R²', y='Modelo', data=results_df, palette='viridis')
    plt.title('Comparação de R²')
    plt.xlabel('Coeficiente de Determinação (R²)')

    # MAE
    plt.subplot(2, 2, 3)
    sns.barplot(x='MAE', y='Modelo', data=results_df, palette='viridis')
    plt.title('Comparação de MAE')
    plt.xlabel('Erro Absoluto Médio (MAE)')

    # Tempo de Treino
    plt.subplot(2, 2, 4)
    sns.barplot(x='Tempo Treino (s)', y='Modelo', data=results_df, palette='viridis')
    plt.title('Tempo de Treinamento')
    plt.xlabel('Segundos')

    plt.tight_layout()
    plt.savefig('comparacao_modelos.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("\n" + "="*80)
    print("COMPARAÇÃO DE MODELOS")
    print("="*80)
    print(results_df.to_string(index=False))

    # Salvar resultados em CSV
    results_df.to_csv('resultados_modelos.csv', index=False)

    return results_df

In [ ]:
# ====================================================
# PASSO: 8 Otimização de Hiperparâmetros
# ====================================================
def optimize_hyperparameters(model, param_grid, preprocessor, X_train, y_train):
    """Otimiza hiperparâmetros usando RandomizedSearchCV"""
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid,
        n_iter=20,
        cv=5,
        scoring='neg_root_mean_squared_error',
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    print(f"\nIniciando otimização para {type(model).__name__}...") # Corrected attribute access
    search.fit(X_train, y_train)

    print(f"Melhores parâmetros encontrados:")
    for param, value in search.best_params_.items():
        print(f"  {param}: {value}")

    return search.best_estimator_, search.best_params_

In [ ]:
# ====================================================
#PASSO: 9 Comparação de Modelos Otimizados
# ====================================================
def compare_optimized_models(best_models, X_test, y_test):
    """Compara o desempenho dos modelos otimizados"""
    results = []

    for name, model in best_models.items():
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        results.append({
            'Modelo': name,
            'RMSE': rmse,
            'R²': r2,
            'MAE': mae
        })

    # Criar DataFrame de resultados
    results_df = pd.DataFrame(results).sort_values('RMSE')

    # Plotar comparação
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    sns.barplot(x='RMSE', y='Modelo', data=results_df, palette='viridis')
    plt.title('Comparação de RMSE (Modelos Otimizados)')
    plt.xlabel('RMSE')

    plt.subplot(1, 2, 2)
    sns.barplot(x='R²', y='Modelo', data=results_df, palette='viridis')
    plt.title('Comparação de R² (Modelos Otimizados)')
    plt.xlabel('Coeficiente de Determinação (R²)')

    plt.tight_layout()
    plt.savefig('comparacao_modelos_otimizados.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("\n" + "="*80)
    print("COMPARAÇÃO DE MODELOS OTIMIZADOS")
    print("="*80)
    print(results_df.to_string(index=False))

    # Salvar resultados em CSV
    results_df.to_csv('resultados_modelos_otimizados.csv', index=False)

    return results_df

In [ ]:
if __name__ == "__main__":
    import pandas as pd
    import joblib
    from scipy.stats import randint, uniform
    import traceback
    import re

    # Carregar dados
    try:
        # Substitua com seu carregamento real de dados
        # df = pd.read_csv('obesity_data.csv')

        # Dados de exemplo otimizados para testes rápidos
        print("Usando dados de exemplo otimizados...")
        data = {
            'YearStart': list(range(2010, 2020)) * 10,
            'YearEnd': list(range(2010, 2020)) * 10,
            'LocationDesc': ['Alabama', 'Alaska', 'Arizona'] * 33 + ['California'],
            'StratificationCategory1': ['Total', 'Gender', 'Education'] * 33 + ['Income'],
            'Data_Value': np.random.uniform(20, 40, 100),
            'Sample_Size': np.random.randint(100, 1000, 100)
        }
        df = pd.DataFrame(data)
        df.rename(columns={'Data_Value': 'obesity_rate'}, inplace=True)

        # Executar o fluxo principal
        main(df)

    except Exception as e:
        print(f"Erro ao carregar dados: {str(e)}")
        traceback.print_exc()

Usando dados de exemplo otimizados...
Realizando limpeza básica de dados...
Colunas originais: 6, Colunas após limpeza: 6
Colunas de data identificadas: ['YearStart', 'YearEnd']
Realizando pré-processamento de dados...
Coluna de data processada: YearStart
Coluna de data processada: YearEnd
Pré-processamento concluído.
Realizando engenharia de features...
Engenharia de features concluída.
Gradient Boosting: RMSE=6.6541, R²=-0.0319, MAE=5.5523, Tempo=0.20s
Random Forest: RMSE=6.4377, R²=0.0341, MAE=5.4328, Tempo=0.31s
Linear Regression: RMSE=6.4092, R²=0.0426, MAE=5.3296, Tempo=0.03s
Support Vector Machine: RMSE=6.8315, R²=-0.0877, MAE=5.5241, Tempo=0.04s
K-Nearest Neighbors: RMSE=6.6964, R²=-0.0451, MAE=5.7166, Tempo=0.03s
ElasticNet: RMSE=6.4104, R²=0.0423, MAE=5.3427, Tempo=0.03s

COMPARAÇÃO DE MODELOS
                Modelo     RMSE        R²      MAE       MSE  Tempo Treino (s)
     Linear Regression 6.409187  0.042640 5.329599 41.077681          0.030302
            ElasticNet 6.41